In [1]:
import pygame
import math

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class environment:
    def __init__(self,dimensions):
        #importing colors
        self.black = (0,0,0)
        self.white = (255,255,255)
        self.green = (0,255,0)
        self.blue = (0,0,255)
        self.red = (255,0,0)
        self.yellow = (255,255,0)
        self.gray = (122, 122, 122)
        #dimensions 
        self.height = dimensions[0]
        self.width = dimensions [1]
        #display window settings
        pygame.display.set_caption("Mobile Robotics Project - Faisal Javed")
        self.map = pygame.display.set_mode((self.width,self.height))
        
        #text variables
        self.font = pygame.font.Font('freesansbold.ttf',30)
        self.text = self.font.render('default',True,self.white,self.black)
        self.textRect = self.text.get_rect()
        self.textRect.center = (dimensions[1]-600, dimensions[0]-100)
        
        self.font1 = pygame.font.Font('freesansbold.ttf',50)
        self.text1 = self.font1.render('default',True,self.white,self.black)
        self.textRect1 = self.text1.get_rect()
        self.textRect1.center = (dimensions[1]-1000, dimensions[0]-575)
        
        #trail history
        self.trail_set = []
        
    def write_info(self,v,v1,theta):
        txt = f"v = {v} v1 = {v1} theta = {int(math.degrees(theta))}"
        self.text = self.font.render(txt,True,self.white,self.black)
        self.map.blit(self.text,self.textRect)
        
        txt1 = f"MOBILE ROBOTICS: FAISAL JAVED "
        self.text1 = self.font1.render(txt1,True,self.white,self.black)
        self.map.blit(self.text1,self.textRect1)
        
    def trail(self,pos):
        for i in range(0,len(self.trail_set)-1):   #will work on first in first out logic, updated at every instant 
            pygame.draw.line(self.map,self.yellow,(self.trail_set[i][0],self.trail_set[i][1]),
                            (self.trail_set[i+1][0],self.trail_set[i+1][1]))
        if self.trail_set.__sizeof__()>30000:
            self.trail_set.pop(0)
        self.trail_set.append(pos)    
        
         
    def robot_frame(self,pos,rotation):    #center point is robot location
        n=80
        
        centerx,centery = pos
        x_axis = (centerx + n*math.cos(-rotation),centery + n*math.sin(-rotation))   #drawing axes of x-y plane
        y_axis = (centerx + n*math.cos(-rotation+math.pi/2),centery + n*math.sin(-rotation+math.pi/2))
        pygame.draw.line(self.map,self.red,(centerx,centery),x_axis,3)
        pygame.draw.line(self.map,self.green,(centerx,centery),y_axis,3)
        
        

In [3]:
class Robot:
    def __init__(self,start,img,width): #take robot start position, image and width
        self.m2p = 3779.52 #meters to pixels conversion factor
        #robot dimensions
        self.w = width
        self.x = start[0] #start position x coordinate
        self.y = start[1]  #start position y coordinate
        self.theta = 0 #theta is heading angle
        self.v = 0.01*self.m2p  #initial velociy in m/sec of left wheel
        self.v1 = 0.01*self.m2p  #initial velociy in m/sec of right wheel
        self.maxspeed = 0.02*self.m2p
        self.minspeed = -0.02*self.m2p
        
        #graphics
        self.img = pygame.image.load(img)
        self.rotated = self.img  #to create temporary vessel
        self.rect = self.rotated.get_rect(center=(self.x,self.y))
        
    def draw(self,map):
        map.blit(self.rotated,self.rect)
        
    def move(self,event=None): #initailly event is none or empty
        if event is not None:   #if any event happens
            if event.type == pygame.KEYDOWN:  #any key is pressed
                if event.key == pygame.K_LEFT:  #if 4 is pressed, increase velocity of left wheel
                    self.v+= 0.001*self.m2p
                elif event.key == pygame.K_DOWN:
                    self.v-= 0.001*self.m2p   #if 1 is pressed, decrease velocity of left wheel
                elif event.key == pygame.K_RIGHT:
                    self.v1+= 0.001*self.m2p   #if 6 is pressed, increase velocity of right wheel    
                elif event.key == pygame.K_UP:
                    self.v1-= 0.001*self.m2p   #if 3 is pressed, decrease velocity of right wheel    
        self.x+=((self.v+self.v1)/2)*math.cos(self.theta)*dt   #mathematical modeling velocityy eq, dt is time difference 
        self.y-=((self.v+self.v1)/2)*math.sin(self.theta)*dt   #minus sign because y axis is opposite for screen and robot 
        self.theta+=(self.v1-self.v)/self.w*dt
        
        #RESET THETA
        if self.theta>2*math.pi or self.theta<-2*math.pi:
            self.theta = 0
        
        #setting max speed 
        self.v1 = min(self.v1,self.maxspeed)
        self.v = min(self.v,self.maxspeed)
        
        #setting min speed 
        self.v1 = max(self.v1,self.minspeed)
        self.v = max(self.v,self.minspeed)
        
        self.rotated = pygame.transform.rotozoom(self.img,math.degrees(self.theta),1)
        self.rect = self.rotated.get_rect(center=(self.x,self.y))
        

In [4]:
        
#initialization of pygame
pygame.init()

#Start Position
start = (200,200)

#dimensions
dims = (600,1200)

#check if its running 
running = True

# callin environment class and giving input dimensions
Environment = environment(dims)

robot = Robot(start,r"C:\Users\dell\robotImg.jpeg",0.01*8000) #start position, image and width, initially 1cm 3779.52 
dt = 0
lasttime = pygame.time.get_ticks()
#simulating loops
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:  #to quit simulaion press stop button
            running = False #to close the window
        robot.move(event)    
    dt = (pygame.time.get_ticks()-lasttime)/1000 #to find time difference of current time and previous loop time in seconds
                                                            #divided by 1000 to covert milli seconds into seconds 
    lasttime = pygame.time.get_ticks()
    
    
    pygame.display.update() #update window at every instant
    Environment.map.fill(Environment.gray)
    robot.move()
    Environment.write_info(int(robot.v),int(robot.v1),int(robot.theta))
    robot.draw(Environment.map)
    Environment.robot_frame((robot.x,robot.y),robot.theta)
    Environment.trail((robot.x,robot.y))
     
    
    
    

KeyboardInterrupt: 